In [1]:
from importlib import reload
import mrf
import AgentS1
import AgentS2
import utils
import evaluate

In [45]:
from pathlib import Path
import json

class RunAgents():
    def __init__(self, question_schema, prompts, log_dir, city, model_dial, model_trans, chat_temp):
        # check if log_dir exists, if not add

        self.question_schema = question_schema
        self.prompts = prompts
        path = Path(log_dir)
        path.mkdir(parents=True, exist_ok=True)

        self.log_dir = log_dir
        self.city = city
        self.model_dial = model_dial
        self.model_trans = model_trans
        self.chat_temp = chat_temp

    
    def run_agents(self):
        self.agents1 = AgentS1.AgentS1(self.prompts, self.question_schema, city=self.city, log_dir=self.log_dir, log_name='agents1.json', model_dial=self.model_dial, model_trans=self.model_trans, chat_temp=self.chat_temp)
        
        s1_schema = self.agents1.compile()

        self.agents2 = AgentS2.AgentS2(self.prompts, s1_schema, city=self.city, log_dir=self.log_dir, log_name='agents2.json', model_dial=self.model_dial, model_trans=self.model_trans, chat_temp=self.chat_temp)

        s2_schema = self.agents2.compile()

        # save s2_schema
        json.dump(s2_schema, open(f'{self.log_dir}/agents.json', 'w'), indent=4)

        return s2_schema




In [60]:
import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from tabulate import tabulate
import numpy as np
import torch
reload(mrf)
reload(AgentS1)
reload(AgentS2)
reload(utils)
reload(evaluate)

class evaluation_suite():
    def __init__(self, question_folder, identifier='', log_dir='./Experiments', prompts='./data/prompts_13.json', model_dial='gpt-4o', model_trans='gpt-3.5', chat_temp=0.3, data_folder='./data/'):

        self.question_folder = question_folder
        if identifier == '':
            self.identifier = str(datetime.datetime.now().strftime('%m%d_%H%M'))
        else:
            self.identifier = identifier

        self.prompts = prompts

        self.log_dir = log_dir

        self.model_dial = model_dial
        self.model_trans = model_trans
        self.chat_temp = chat_temp
        self.data_folder = data_folder

        self.agent_records = {}
        self.exp_records = {}



    def run_agent(self, ):

        # load questions from folder
        question_folder = Path(self.question_folder)
        question_files = list(question_folder.glob('*.json'))
        for question in question_files:
            question_schema = json.load(open(question))


            city = question_schema['City']
            question_schema.pop('City')
            # remove state after ", "
            question_name = city.split(',')[0].replace(' ', '')

            run_agents = RunAgents(question_schema=question_schema, prompts=self.prompts, log_dir=self.log_dir+f'/{question_name}', city=city, model_dial=self.model_dial, model_trans=self.model_trans, chat_temp=self.chat_temp)

            s2_schema_records = run_agents.run_agents()

            self.agent_records[question_name] = s2_schema_records
            

    def load_experiments(self, ):
        # load experiments *_schema.json
        question_folder = Path(self.log_dir)
        # get all folders under question folder, not files
        question_folders = [x for x in question_folder.iterdir() if x.is_dir() and '.' not in x.stem]
        for question in question_folders:
            question_name = question.stem
            schema = json.load(open(f'{question}/agents2.json'))
            self.agent_records[question_name] = schema
            print(schema['schema']['Variables'])


    def optimize_mrf(self, w0=0, update_iter=50000):

        for question, schema in self.agent_records.items():
            print(f'Evaluating {question}')
            print(schema['schema'])
            mrf_obj = mrf.Brute(verbose=True)
            #tmp = schema['schema'].copy()
            #tmp = remove_vars(tmp, [], one='Price')
            mrf_obj.from_json(schema['schema'])
            #print(schema['schema']['Constraints'])
            #mrf_obj.from_json(tmp)
            if w0 != 0:
                mrf_obj.set_w0(w0)
            mrf_obj.update(update_iter, slack=False)

            # save model to same directory
            torch.save(mrf_obj, f'{self.log_dir}/{question}/ent{str(w0)}.pt')
        

    def get_results(self, w0=0, ):
        
        for question, schema in self.agent_records.items():

            print(f'Evaluating {question}')
            try:
                mrf_obj = torch.load(f'{self.log_dir}/{question}/ent{str(w0)}.pt')
                mrf_result = mrf_obj.query(schema['schema'])
            except:
                mrf_result = 0
            zero_result = schema['zero shot']['result']['Probability']
            cot_result = schema['cot']['result']['Probability']
            # data folder
            dat_path = Path(self.data_folder + '/' + question + '.csv')
            data = utils.DatasetQ(dat_path)
            ground_truth = data.marg(schema['schema']['Question'])
            self.exp_records[question] = {'mrf': mrf_result, 'zero_shot': zero_result, 'cot': cot_result, 'ground_truth': ground_truth}
            
            json.dump(self.exp_records[question], open(f'{self.log_dir}/{question}/run_results{str(w0)}.json', 'w'), indent=4)



    def evaluate(self,):

        table = [["City", "Zero Shot", "Ground", "MRF", "CoT", "Zero TVD", "MRF TVD", "CoT TVD"]]

        for city, results in self.exp_records.items():
            tmp = []
            #print(results)
            mrf_ground = evaluate.total_variation_distance(results['ground_truth'], results['mrf'])
            zero_shot_ground = evaluate.total_variation_distance(results['ground_truth'], results['zero_shot'])
            cot_ground = evaluate.total_variation_distance(results['ground_truth'], results['cot'])
            tmp.append(city)
            tmp.append(np.array(results['zero_shot']).round(3))
            tmp.append(np.array(results['ground_truth']).round(3))
            tmp.append(np.array(results['mrf']).round(3))
            tmp.append(np.array(results['cot']).round(3))
            #tmp.append((mrf_ground - zero_shot_ground).round(3))
            tmp.append(zero_shot_ground.round(3))
            tmp.append(mrf_ground.round(3))
            tmp.append(cot_ground.round(3))
            table.append(tmp)

        print(tabulate(table))
        return table


test = evaluation_suite(question_folder='./test_questions2/LosAngeles_0', identifier='test', log_dir='./Experiments2', prompts='./data/prompts_13.json', model_dial='gpt-4o', model_trans='gpt-3.5-turbo', chat_temp=0)


"""
# How to use
question_folder: folder with json question files. If you run this script only to get one run on one city, separate the questions so question_folder is './Questions/BrowardCounty' and your question file like './Questions/BrowardCounty/question.json', no restriction on the file name


identifier: ignore this


log_dir: directory to save results. Similarly if one run one city, set log_dir to './Experiments/run_1'. Script will create and populate folder './Experiments/run_1/BrowardCounty'


prompts: fixed path to the prompts file, it contains all prompts needed



data_folder: fixed path to the data folder. if set to './data/', will look for csv './data/BrowardCounty.csv'


the rest are intuitive
"""



# This will iterate over all questions, one in this case, including cot and zero shot, and save the results in the log_dir/cityname/ folder
test.run_agent()


# load the saved experiment files that resulted from the previous step
test.load_experiments()


# this will optimize the MRF model, w0=0 will be faster, automatically save the model to the same folder log_dir/cityname/
test.optimize_mrf(w0=0.4, update_iter=10000)



# get results, this only attempts to load mrf from local. If it doesn't exist, it will record other results and dump in log_dir/cityname/
test.get_results(w0=0.4)



# this will print the tabulate
ret = test.evaluate()


In [24]:
def remove_vars(constraints, vars, one=''):
    ret = {}
    ret['Variables'] = []
    ret['Constraints'] = []
    check = set()
    for var in constraints['Variables']:
        if var['Name'] not in vars:
            ret['Variables'].append(var)
    for cons in constraints['Constraints']:
        if cons is None:
            print("None")
            continue
        if cons['Target'][0]['Name'] not in vars and (len(cons['Condition']) == 0 or (cons['Condition'][0]['Name'] not in vars)):
            if len(cons['Condition']) != 0 or cons['Target'][0]['Name'] != one:
                print(cons['Target'][0]['Name'], cons['Condition'])
                tt = {}
                tt['Target'] = cons['Target']
                tt['Probability'] = cons['Probability']
                #tt['Question'] = cons['Question']
                tt['Condition'] = []
                for cond in cons['Condition']:
                    if cond['Name'] == 'City':
                        continue
                    tt['Condition'].append(cond)
                ret['Constraints'].append(tt)
            else:
                print("here")
    return ret